In [1]:
import chess.pgn
import pandas as pd
import chess
import numpy as np
from scipy import linalg

In [2]:
ls -shr

total 90M
 16K README.md                              568K chess.ipynb
 89M lichess_db_standard_rated_2013-01.pgn   56K chess-and-eigenvectors.ipynb


In [3]:
board = chess.Board()

In [4]:
pgn = open('lichess_db_standard_rated_2013-01.pgn',encoding="utf-8")

In [5]:
#creates a list with one entry for each space on the board
spaces = []
for i in range(1,9):
    for j in range (1,9):
        spaces.append(chr(j+96)+str(i))

In [6]:
#create a matrix to tally weights for every pair of opposing pieces
pieces_list = [space for space in spaces if any(map(space.__contains__, ['1','2','7','8']))]
weights = pd.DataFrame(0,index=pieces_list, columns=pieces_list)

In [7]:
#create a state dictionary of each square and the initial square of each piece
#Those 'None' entries are the empty squares in the middle of the board, before the first move
#Each piece is on it's own square to begin
state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}
state_dict

{'a1': 'a1',
 'b1': 'b1',
 'c1': 'c1',
 'd1': 'd1',
 'e1': 'e1',
 'f1': 'f1',
 'g1': 'g1',
 'h1': 'h1',
 'a2': 'a2',
 'b2': 'b2',
 'c2': 'c2',
 'd2': 'd2',
 'e2': 'e2',
 'f2': 'f2',
 'g2': 'g2',
 'h2': 'h2',
 'a3': None,
 'b3': None,
 'c3': None,
 'd3': None,
 'e3': None,
 'f3': None,
 'g3': None,
 'h3': None,
 'a4': None,
 'b4': None,
 'c4': None,
 'd4': None,
 'e4': None,
 'f4': None,
 'g4': None,
 'h4': None,
 'a5': None,
 'b5': None,
 'c5': None,
 'd5': None,
 'e5': None,
 'f5': None,
 'g5': None,
 'h5': None,
 'a6': None,
 'b6': None,
 'c6': None,
 'd6': None,
 'e6': None,
 'f6': None,
 'g6': None,
 'h6': None,
 'a7': 'a7',
 'b7': 'b7',
 'c7': 'c7',
 'd7': 'd7',
 'e7': 'e7',
 'f7': 'f7',
 'g7': 'g7',
 'h7': 'h7',
 'a8': 'a8',
 'b8': 'b8',
 'c8': 'c8',
 'd8': 'd8',
 'e8': 'e8',
 'f8': 'f8',
 'g8': 'g8',
 'h8': 'h8'}

In [8]:
game_counter = 0
total_move_counter = 0
game = chess.pgn.read_game(pgn)

while game is not None:
    counter = 0
    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

    #print(game)
    #do stuff here



    #for each line in the file
    for move in game.mainline_moves():
        counter += 1
        if board.is_pseudo_legal(move):


            if board.is_capture(move):
                capturing_piece = state_dict[chess.square_name(move.from_square)]
                captured_piece = state_dict[chess.square_name(move.to_square)]
                #print('capturing piece',capturing_piece)
                #print('captured piece',captured_piece)

                if board.is_en_passant(move):
                    #do en passant stuff here
                    #print("en passant")
                    pass
                elif capturing_piece is not None:
                    if captured_piece is not None:

                        #record the kill in a matrix of all kills
                        #print(counter,chess.square_name(move.from_square), "takes on",chess.square_name(move.to_square))
                        #print(counter,state_dict[chess.square_name(move.from_square)], "takes",state_dict[chess.square_name(move.to_square)])

                        #what piece is there now?
                     

                        #increment an entry in the weights matrix when one piece takes another
                        weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[chess.square_name(move.to_square)]] += 1

                        #update the piece on the target square
                        state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                        #update the source square to be empty
                        state_dict[chess.square_name(move.from_square)] = None

                        #board.push(move)  # Make the move


                        #if df.loc[i+1,'move_no'] == 1:
                        #    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

            else:


                #add castling special cases to record both moves
                if board.is_kingside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles kingside')
                        #move the appopriate rook
                        state_dict['f1'] = 'h1'
                        #leaving the rook source square empty
                        state_dict['h1'] = 'None'
                        #board.push(move)  # Make the move



                    elif not board.turn:
                        #print(counter,'... and black castles kingside')
                        #move the appopriate rook
                        state_dict['f8'] = 'h8'
                        #leaving the rook source square empty
                        state_dict['h8'] = 'None'
                        #board.push(move)  # Make the move

                elif board.is_queenside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles queenside')
                        #move the appopriate rook
                        state_dict['d1'] = 'a1'
                        #leaving the rook source square empty
                        state_dict['a1'] = 'None'
                        #board.push(move)  # Make the move

                    elif not board.turn:
                        #print(counter,'... and black castles queenside')
                        #move the appopriate rook
                        state_dict['d8'] = 'a8'
                        #leaving the rook source square empty
                        state_dict['a8'] = 'None'
                        #board.push(move)  # Make the move

                else:
                    #update the piece on the target square
                    state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                    #update the source square to be empty
                    state_dict[chess.square_name(move.from_square)] = None


                    #print(counter, chess.square_name(move.from_square),chess.square_name(move.to_square))

        board.push(move)  # Make the move
        #print(state_dict)
            
    if board.is_checkmate():
        checkers_list = [state_dict[chess.square_name(x)] for x in list(board.checkers())]
        if checkers_list[0] is not None:
            piece_giving_check = str(checkers_list[0])

            if board.turn:
                captured_king = 'e1'
            else:
                captured_king = 'e8'
            weights.loc[piece_giving_check,captured_king] += 1

    
    game_counter += 1
    #print('end of game',game_counter)
    
    total_move_counter += counter
    #print('total moves analyzed',total_move_counter)

    
    #print('board outcome',board.outcome())


    #print('is game over?',board.is_game_over())
    board.reset()

    game = chess.pgn.read_game(pgn)




    #print(state_dict)

    

KeyboardInterrupt: 

In [9]:
print(game_counter,'games analyzed')
print(total_move_counter,'moves analyzed')

17203 games analyzed
1128746 moves analyzed


In [10]:
weights

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,g7,h7,a8,b8,c8,d8,e8,f8,g8,h8
a1,0,7,7,11,0,1,3,14,24,19,...,358,386,1769,535,632,695,313,598,465,1345
b1,6,0,8,2,1,5,6,10,9,7,...,230,180,541,772,921,638,43,1338,1827,415
c1,17,12,0,15,0,9,10,16,23,19,...,554,440,390,1264,1,582,39,2937,2854,714
d1,16,7,8,0,1,5,7,14,17,10,...,902,865,1182,1764,2195,3752,1327,1578,1570,1156
e1,0,0,1,1,0,0,0,0,0,0,...,76,83,137,147,186,352,0,221,214,180
f1,8,5,6,9,0,0,6,15,5,10,...,262,384,589,2574,2948,460,45,0,1538,425
g1,14,7,12,5,1,4,0,13,9,9,...,456,325,320,2207,1157,590,31,972,760,573
h1,15,9,8,8,1,5,3,0,26,17,...,559,579,1409,571,730,808,320,738,655,1666
a2,9,1,3,1,0,2,2,1,0,11,...,19,23,48,231,130,82,61,96,43,30
b2,5,5,1,2,0,4,2,2,22,0,...,11,9,63,213,87,81,45,997,320,22


In [11]:
a = weights - np.identity(32)
print(a)

        a1      b1      c1      d1      e1      f1      g1      h1     a2  \
a1    -1.0     7.0     7.0    11.0     0.0     1.0     3.0    14.0   24.0   
b1     6.0    -1.0     8.0     2.0     1.0     5.0     6.0    10.0    9.0   
c1    17.0    12.0    -1.0    15.0     0.0     9.0    10.0    16.0   23.0   
d1    16.0     7.0     8.0    -1.0     1.0     5.0     7.0    14.0   17.0   
e1     0.0     0.0     1.0     1.0    -1.0     0.0     0.0     0.0    0.0   
f1     8.0     5.0     6.0     9.0     0.0    -1.0     6.0    15.0    5.0   
g1    14.0     7.0    12.0     5.0     1.0     4.0    -1.0    13.0    9.0   
h1    15.0     9.0     8.0     8.0     1.0     5.0     3.0    -1.0   26.0   
a2     9.0     1.0     3.0     1.0     0.0     2.0     2.0     1.0   -1.0   
b2     5.0     5.0     1.0     2.0     0.0     4.0     2.0     2.0   22.0   
c2     3.0     0.0     0.0     2.0     1.0     5.0     0.0     5.0    8.0   
d2     1.0     2.0     3.0     1.0     0.0     2.0     3.0     7.0   10.0   

In [12]:
b = np.zeros(32)
print(b)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [13]:
x = linalg.solve(a, b)
x

array([ 0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,
       -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,
       -0., -0., -0., -0.,  0.,  0.])

### Why doesn't it work?
### Because the solver is finding the trivial solution. i.e. just a vector full of zeroes.
### To check that there is actually a non trivial solution, the determinant of A must be non-zero.

In [14]:
np.linalg.det(a)

-3.648322519445746e+91

### OK since the determinant is not zero, there must be at least one non-trivial solution. How to find it?
### An homogenous linear equation can be solved with a Matrix Factorization.

In [15]:
eigen_values, eigen_vectors = np.linalg.eig(a)  

In [16]:
values = eigen_vectors[:, np.argmin(eigen_values)] 

In [17]:
## Let's normalize the values

In [18]:
list = []
for i in values:
    list.append(abs(float(i)))

<ipython-input-18-fd238d3eb4fa>:3: ComplexWarning: Casting complex values to real discards the imaginary part
  list.append(abs(float(i)))


In [23]:
#The fractional inverse of the average of all the pawns
factor = 1 / np.mean(list[8:24])

In [24]:
normed = [i*factor for i in list]

In [25]:
normalized_list = ["%.2f"%i for i in normed]

In [26]:
print(normalized_list[24:36])
print(normalized_list[16:24])
print(normalized_list[8:16])
print(normalized_list[0:8])


['3.19', '3.53', '4.07', '8.18', '1.02', '4.35', '3.53', '3.17']
['0.46', '1.17', '1.20', '1.34', '1.47', '1.08', '0.94', '0.53']
['0.51', '1.06', '1.07', '1.31', '1.42', '1.04', '0.91', '0.50']
['3.10', '3.49', '4.29', '8.37', '0.82', '4.22', '3.67', '3.40']


In [ ]:
board.reset()
game = chess.pgn.read_game(pgn)

for move in game.mainline_moves():
        print(move.uci())
        print('turn?' , board.turn)
        print('is game over?' , board.is_game_over())
        print('from square: ' ,chess.square_name(move.from_square))
        print('to square: ' ,chess.square_name(move.to_square))
        print('drop: ' ,move.drop)
        print('is capture?' , board.is_capture(move))
        print('is en passant capture?' , board.is_en_passant(move))
        print('is kingside castle?' , board.is_kingside_castling(move))
        print('is quenside castle?' , board.is_queenside_castling(move))

        board.push(move)  # Make the move
        print('is_checkmate?' , board.is_checkmate())
        #print('checkers' , board.checkers())
        print('outcome' , board.outcome())

        #print(board)
        #print(board.piece_map())
    #board

In [ ]:
U, D, V = np.linalg.svd(a, full_matrices=True)